In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import RobustScaler

In [7]:
df = pd.read_csv('C:/Users/lyc5d/Desktop/DIC-Airfare-Prediction/dataset/cleaned_data.csv')
df.columns = df.columns.str.replace('[^a-zA-Z0-9_]', '_', regex=True)
df.drop(columns=['price', 'idx'], errors='ignore')

df.columns.to_list()
df

,idx,airline,flight_code,source_city,time_taken,stop,destinate_city,price,Class,Days_Left,dep_time_category,arr_time_category
0,0,SpiceJet,SG-8709,Delhi,130,non-stop,Mumbai,5953,Economy,1,Evening,Night
1,1,SpiceJet,SG-8157,Delhi,140,non-stop,Mumbai,5953,Economy,1,Early Morning,Morning
2,2,AirAsia,I5-764,Delhi,130,non-stop,Mumbai,5956,Economy,1,Early Morning,Early Morning
3,3,Vistara,UK-995,Delhi,135,non-stop,Mumbai,5955,Economy,1,Morning,Afternoon
4,4,Vistara,UK-963,Delhi,140,non-stop,Mumbai,5955,Economy,1,Morning,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...
298921,301051,Vistara,UK-822,Chennai,605,1-stop,Hyderabad,69265,Business,49,Morning,Evening
298922,301052,Vistara,UK-826,Chennai,625,1-stop,Hyderabad,77105,Business,49,Afternoon,Night
298923,301053,Vistara,UK-832,Chennai,830,1-stop,Hyderabad,79099,Business,49,Early Morning,Night
298924,301054,Vistara,UK-828,Chennai,600,1-stop,Hyderabad,81585,Business,49,Early Morning,Evening


In [54]:
from sklearn.preprocessing import LabelEncoder

lb_encode = LabelEncoder()
df['airline'] = lb_encode.fit_transform(df['airline'])
df['flight_code'] = lb_encode.fit_transform(df['flight_code'])
df['source_city'] = lb_encode.fit_transform(df['source_city'])
df['time_taken'] = lb_encode.fit_transform(df['time_taken'])
df['stop'] = lb_encode.fit_transform(df['stop'])
df['destinate_city'] = lb_encode.fit_transform(df['destinate_city'])
df['Class'] = lb_encode.fit_transform(df['Class'])
df['dep_time_category'] = lb_encode.fit_transform(df['dep_time_category'])
df['arr_time_category'] = lb_encode.fit_transform(df['arr_time_category'])
df

,Unnamed__0,airline,flight_code,source_city,time_taken,stop,destinate_city,price,Class,Days_Left,dep_time_category,arr_time_category
0,0,4,1721,2,19,2,5,5953,1,1,2,4
1,1,4,1698,2,21,2,5,5953,1,1,1,3
2,2,1,1497,2,19,2,5,5956,1,1,1,1
3,3,7,1966,2,20,2,5,5955,1,1,3,0
4,4,7,1946,2,21,2,5,5955,1,1,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
298921,301051,7,1820,1,116,0,3,69265,0,49,3,2
298922,301052,7,1827,1,120,0,3,77105,0,49,0,4
298923,301053,7,1836,1,162,0,3,79099,0,49,1,4
298924,301054,7,1831,1,115,0,3,81585,0,49,1,2


In [55]:
from sklearn.model_selection import train_test_split
X = df.drop(axis=1, columns=['price'])
y = df.price
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from matplotlib.pylab import rcParams
# rcParams['figure.figsize'] = 12, 4
# get_ipython().run_line_magic('matplotlib', 'inline')

kfolds = KFold(n_splits=50, shuffle=True, random_state=100)

def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=kfolds))

    return rmse

xgb3 = XGBRegressor(learning_rate=0.5, 
                    n_estimators=300, 
                    max_depth=6, 
                    min_child_weight=10, 
                    gamma=0, 
                    subsample=0.8, 
                    max_bin=128,
                    colsample_bytree=0.8, 
                    objective='reg:linear', 
                    nthread=4, 
                    scale_pos_weight=1, 
                    seed=27, 
                    reg_alpha=0.0001,
                    tree_method='gpu_hist',
                    device='cuda')

xgb_fit = xgb3.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)
cv_rmse(xgb_fit).mean()

[0]	validation_0-rmse:12193.90598
[1]	validation_0-rmse:7467.50736
[2]	validation_0-rmse:5405.55707
[3]	validation_0-rmse:4646.17222
[4]	validation_0-rmse:4339.18893
[5]	validation_0-rmse:4163.15086
[6]	validation_0-rmse:4035.25784


c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:50:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:50:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[7]	validation_0-rmse:3991.61119
[8]	validation_0-rmse:3939.22009
[9]	validation_0-rmse:3848.09411
[10]	validation_0-rmse:3760.50679
[11]	validation_0-rmse:3748.21804
[12]	validation_0-rmse:3681.12391
[13]	validation_0-rmse:3660.79393
[14]	validation_0-rmse:3597.82405
[15]	validation_0-rmse:3554.15168
[16]	validation_0-rmse:3547.80691
[17]	validation_0-rmse:3525.86781
[18]	validation_0-rmse:3494.65326
[19]	validation_0-rmse:3474.61194
[20]	validation_0-rmse:3431.67907
[21]	validation_0-rmse:3401.22190
[22]	validation_0-rmse:3365.60692
[23]	validation_0-rmse:3333.72142
[24]	validation_0-rmse:3317.98806
[25]	validation_0-rmse:3304.91555
[26]	validation_0-rmse:3275.20323
[27]	validation_0-rmse:3266.97882
[28]	validation_0-rmse:3244.93081
[29]	validation_0-rmse:3238.67995
[30]	validation_0-rmse:3217.50462
[31]	validation_0-rmse:3201.76447
[32]	validation_0-rmse:3177.21981
[33]	validation_0-rmse:3169.37872
[34]	validation_0-rmse:3149.16117
[35]	validation_0-rmse:3139.47826
[36]	validation_0

c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:50:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:50:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:50:40] WARNING: C:\buildkite-agent\builds\b

np.float64(2418.2415666000484)

In [57]:
from lightgbm import LGBMRegressor

lgbm_model = LGBMRegressor(objective='regression', 
                           num_leaves=31, 
                           learning_rate=0.05, 
                           n_estimators=300,
                           max_bin=128, 
                           bagging_fraction=0.8, 
                           bagging_freq=5, 
                           feature_fraction=0.8,
                           feature_fraction_seed=9, 
                           bagging_seed=9,
                           min_data_in_leaf=50, 
                           min_sum_hessian_in_leaf=10,
                           reg_alpha=0.1,
                           reg_lambda=0.1)

lgbm_fit = lgbm_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
cv_rmse(lgbm_model).mean()

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightG

np.float64(3208.080327657103)

In [58]:
from sklearn.ensemble import RandomForestRegressor

forest= RandomForestRegressor(n_estimators=300, 
                              random_state=42,
                              max_depth=20,
                              min_samples_split=10,
                              min_samples_leaf=5,
                              n_jobs=-1,
                              max_features='sqrt')
forest_fit = forest.fit(X_train, y_train)
cv_rmse(forest_fit).mean()

np.float64(2486.232010341147)

In [60]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.pipeline import make_pipeline

lightgbm = make_pipeline(RobustScaler(),
                         LGBMRegressor(objective='regression', 
                           num_leaves=31, 
                           learning_rate=0.05, 
                           n_estimators=300,
                           max_bin=128, 
                           bagging_fraction=0.8, 
                           bagging_freq=5, 
                           feature_fraction=0.8,
                           feature_fraction_seed=9, 
                           bagging_seed=9,
                           min_data_in_leaf=50, 
                           min_sum_hessian_in_leaf=10,
                           reg_alpha=0.1,
                           reg_lambda=0.1))

xgboost = make_pipeline(RobustScaler(),
                        XGBRegressor(learning_rate=0.5, 
                    n_estimators=300, 
                    max_depth=6, 
                    min_child_weight=10, 
                    gamma=0, 
                    subsample=0.8, 
                    max_bin=128,
                    colsample_bytree=0.8, 
                    objective='reg:linear', 
                    nthread=4, 
                    scale_pos_weight=1, 
                    seed=27, 
                    reg_alpha=0.0001,
                    tree_method='gpu_hist',
                    device='cuda'))

forest = make_pipeline(RobustScaler(),
                       RandomForestRegressor(n_estimators=300, 
                              random_state=42,
                              max_depth=20,
                              min_samples_split=10,
                              min_samples_leaf=5,
                              n_jobs=-1,
                              max_features='sqrt'))

stack_gen = StackingCVRegressor(regressors=(lightgbm, xgboost, forest),
                                meta_regressor=xgboost, use_features_in_secondary=True)

stackX = np.array(X_train)
stacky = np.array(y_train)

stack_gen_model = stack_gen.fit(stackX, stacky)

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightG

c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:58:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:58:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:58:18] WARNING: C:\buildkite-agent\builds\b

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightG

c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:58:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:58:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [61]:
stack_gen_preds = stack_gen_model.predict(X_test)
xgb_preds = xgb_fit.predict(X_test)
lgbm_preds = lgbm_fit.predict(X_test)
forest_preds = forest_fit.predict(X_test)

c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:58:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but Robus

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:58:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\lyc5d\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:58:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


In [62]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse_forest = np.sqrt(mean_squared_error(y_test, forest_preds))
print('RMSE_FOREST: %f' % (rmse_forest))

rmse_xgb = np.sqrt(mean_squared_error(y_test, xgb_preds))
print('RMSE_XGB: %f' % (rmse_xgb))

rmse_lgbm = np.sqrt(mean_squared_error(y_test, lgbm_preds))
print('RMSE_LGBM: %f' % (rmse_lgbm))

rmse_stack = np.sqrt(mean_squared_error(y_test, stack_gen_preds))
print('RMSE_STACK: %f' % (rmse_stack))

RMSE_FOREST: 2536.433565
RMSE_XGB: 2451.952009
RMSE_LGBM: 3206.903998
RMSE_STACK: 2318.827177


In [ ]:
from sklearn.metrics import r2_score
print('r2_xgb' ,r2_score(y_test,xgb_preds) )
print('r2_lgbm' , r2_score(y_test,lgbm_preds) )
print('r2_forest' , r2_score(y_test,forest_preds) )
print('r2_stack' , r2_score(y_test,stack_gen_preds) )

abs_error = abs(stack_gen_preds - y_test)
made = 100*(abs_error / y_test)
accuracy = 100-np.mean(made)

print('Accuracy:', round(accuracy, 2), '%')

r2_xgb 0.9882699251174927
r2_lgbm 0.9799346012227814
r2_forest 0.9874477130072891
r2_stack 0.989509105682373
Accuracy: 92.11 %


In [64]:
import pickle as pkl
model = 'C:/Users/lyc5d/Desktop/DIC-Airfare-Prediction/UI/final_model.pkl'
pkl.dump(forest, open(model, 'wb'))